# IBM Applied Data Science Capstone Course by Coursera
## **Week 5 Final Report**
#### *Opening a New Shopping Mall in Kuala Lumpur, Malaysia*

* Build a dataframe of neighborhoods in Kuala Lumpur, Malaysia by web scraping the data from Wikipedia page
* Get the geographical coordinates of the neighborhoods
* Obtain the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new shopping mall

## 1. Import Libraries

In [33]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


## 2. Scrap data from wikipedia

In [0]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text

In [0]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [0]:
# create a list to store neighborhood data
neighborhoodList = []

In [0]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [26]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [27]:
# print the number of rows of the dataframe
kl_df.shape

(71, 1)

##  3. Get the geographical coordinates

In [0]:
# Creating Object

geolocator = Nominatim(user_agent="specify_your_app_name_here")

not_found = []

# define a function to get coordinates
def get_latlng(neighborhood):
    
    try:
      g = geolocator.geocode(f"{neighborhood}, Kuala Lumpur, Malaysia")
      return [g.latitude,g.longitude]
    
    except:
      not_found.append(neighborhood)
      return [np.nan,np.nan]

In [0]:

# call the function to get the coordinates, store in a new list using list comprehension
coords = [get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist()]


In [63]:
coords

[[3.0635703, 101.7389742],
 [3.1521349, 101.7322499],
 [3.19413635, 101.633639612348],
 [3.1002055, 101.7181069],
 [3.07609695, 101.711446759907],
 [3.087314, 101.7205857],
 [3.13083, 101.66944],
 [3.13083, 101.66944],
 [3.11088395, 101.666964072311],
 [nan, nan],
 [3.2018234, 101.6710223],
 [3.1288572, 101.6845528],
 [3.1471068, 101.7086011],
 [3.0584527, 101.6874386],
 [3.158462, 101.6360029],
 [3.15621045, 101.704823638895],
 [3.1300814, 101.69638],
 [3.1646734, 101.6815526],
 [3.107178, 101.71649],
 [3.16740825, 101.698373898462],
 [3.151148, 101.657635],
 [nan, nan],
 [3.1320545, 101.6368473],
 [3.15668595, 101.701975329096],
 [3.0841851, 101.7035521],
 [3.1398496, 101.6785346],
 [nan, nan],
 [3.1337292, 101.7217644],
 [3.1754152, 101.6767475],
 [3.2334772, 101.6604308],
 [3.1649491, 101.7068594],
 [3.168953, 101.72597],
 [nan, nan],
 [3.20280985, 101.637395],
 [3.1181311, 101.673658425232],
 [3.0894376, 101.6846759],
 [3.10636955, 101.669918692623],
 [3.1232944, 101.7269383],
 [3

In [64]:
not_found

['Batu 11 Cheras',
 'Damansara Town Centre',
 'Happy Garden',
 'Kampung Padang Balang',
 'Taman Cheras Hartamas',
 'Taman Taynton View']

In [0]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [0]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [71]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.063570,101.738974
1,"Ampang, Kuala Lumpur",3.152135,101.732250
2,Bandar Menjalara,3.194136,101.633640
3,Bandar Sri Permaisuri,3.100205,101.718107
4,Bandar Tasik Selatan,3.076097,101.711447
5,Bandar Tun Razak,3.087314,101.720586
6,Bangsar,3.130830,101.669440
7,Bangsar Park,3.130830,101.669440
8,Bangsar South,3.110884,101.666964
9,Batu 11 Cheras,NaN,NaN


In [0]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

## 4. Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [92]:
# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516636, 101.6943028.


In [99]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to map
for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    if not np.isnan(lat):
      label = '{}'.format(neighborhood)
      label = folium.Popup(label, parse_html=True)
      folium.CircleMarker(
          [lat, long],
          radius=5,
          popup=label,
          color='blue',
          fill=True,
          fill_color='#3186cc',
          fill_capacity=0.7).add_to(map_kl)  
    
map_kl

In [0]:
# save the map as HTML file
map_kl.save('map_kl.html')

## 5. Use the Foursquare API to explore the neighborhoods

In [109]:
# define Foursquare Credentials and Version
CLIENT_ID = 'T0PUAC05TSFUJOTFSCGGOCS4YJJJ500VSH501NR3KDPARVAH'
CLIENT_SECRET = 'WMDJODFPIIVUABRWBF2FPR2FB0LMOUJPF4Y2TGZZQJDF05BA'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T0PUAC05TSFUJOTFSCGGOCS4YJJJ500VSH501NR3KDPARVAH
CLIENT_SECRET:WMDJODFPIIVUABRWBF2FPR2FB0LMOUJPF4Y2TGZZQJDF05BA


In [113]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()
    try:
      results = results["response"]["groups"][0]['items']
      # return only relevant information for each nearby venue
      for venue in results:
          venues.append([
              neighborhood,
              lat, 
              long, 
              venue['venue']['name'], 
              venue['venue']['location']['lat'], 
              venue['venue']['location']['lng'],  
              venue['venue']['categories'][0]['name']])
    except:
      print (results)

{'meta': {'code': 400, 'errorType': 'param_error', 'errorDetail': 'll must be of the form XX.XX,YY.YY (received nan,nan)', 'requestId': '5d8600baa30619002c613ca5'}, 'response': {}}
{'meta': {'code': 400, 'errorType': 'param_error', 'errorDetail': 'll must be of the form XX.XX,YY.YY (received nan,nan)', 'requestId': '5d8600c0d03360002c63f542'}, 'response': {}}
{'meta': {'code': 400, 'errorType': 'param_error', 'errorDetail': 'll must be of the form XX.XX,YY.YY (received nan,nan)', 'requestId': '5d8600c3446ea6002c76cf55'}, 'response': {}}
{'meta': {'code': 400, 'errorType': 'param_error', 'errorDetail': 'll must be of the form XX.XX,YY.YY (received nan,nan)', 'requestId': '5d8600c62b274a003853caae'}, 'response': {}}
{'meta': {'code': 400, 'errorType': 'param_error', 'errorDetail': 'll must be of the form XX.XX,YY.YY (received nan,nan)', 'requestId': '5d8600d349cf9300373b1cba'}, 'response': {}}
{'meta': {'code': 400, 'errorType': 'param_error', 'errorDetail': 'll must be of the form XX.XX

In [114]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6484, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.06357,101.738974,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.06357,101.738974,Alam Damai Recreational Park,3.066424,101.740265,Other Great Outdoors
2,Alam Damai,3.06357,101.738974,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
3,Alam Damai,3.06357,101.738974,Ivy Sekinchan Seafood Noodle House 适耕莊特制魚丸海鲜面,3.065749,101.748718,Noodle House
4,Alam Damai,3.06357,101.738974,Bukit Besi Hiking Trail,3.073725,101.733300,Trail


In [115]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,100,100,100,100,100,100
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


In [116]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 295 uniques categories.


In [117]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Supplement Shop', 'Other Great Outdoors', 'Noodle House', 'Trail',
       'Park', 'Restaurant', 'Chinese Restaurant',
       'Chinese Breakfast Place', 'Breakfast Spot',
       'Vegetarian / Vegan Restaurant', 'Bubble Tea Shop', 'Multiplex',
       'Coffee Shop', 'Bakery', 'Café', 'Gym / Fitness Center',
       'Dim Sum Restaurant', 'Cantonese Restaurant', 'Convenience Store',
       'Food Court', 'Indian Restaurant', 'Malay Restaurant',
       'Japanese Restaurant', 'Pool Hall', 'Food Truck', 'Snack Place',
       'Steakhouse', 'Sandwich Place', 'Pizza Place', 'Dessert Shop',
       'Middle Eastern Restaurant', 'Pet Store', 'Pharmacy',
       'Burger Joint', 'Thai Restaurant', 'Farmers Market',
       'Padangnese Restaurant', 'Diner', 'Asian Restaurant',
       'Fast Food Restaurant', 'Hakka Restaurant', 'Hotel',
       'Mamak Restaurant', 'Bed & Breakfast', 'Seafood Restaurant',
       'Resort', 'Italian Restaurant', 'Golf Course', 'Supermarket',
       'French Restaurant'], 

In [118]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

## 6. Analyze Each Neighborhood

In [119]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6484, 296)


,Neighborhoods,Accessories Store,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Business Service,Café,Camera Store,Campground,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,City,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Padangnese Restaurant,Palace,Park,Parking,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,TV Station,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Re

In [120]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(65, 296)


,Neighborhoods,Accessories Store,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Business Service,Café,Camera Store,Campground,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,City,Clothing Store,Club House,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Shop,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Padangnese Restaurant,Palace,Park,Parking,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,TV Station,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Tunnel,Turkish Re

In [121]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

40

In [0]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [123]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.00
1,"Ampang, Kuala Lumpur",0.01
2,Bandar Menjalara,0.01
3,Bandar Sri Permaisuri,0.00
4,Bandar Tasik Selatan,0.01


## 7. Cluster Neighborhoods

In [124]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 2, 0, 2, 1, 1, 0, 2], dtype=int32)

In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [126]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.00,2
1,"Ampang, Kuala Lumpur",0.01,0
2,Bandar Menjalara,0.01,0
3,Bandar Sri Permaisuri,0.00,2
4,Bandar Tasik Selatan,0.01,0


In [127]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(65, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,2,3.063570,101.738974
1,"Ampang, Kuala Lumpur",0.01,0,3.152135,101.732250
2,Bandar Menjalara,0.01,0,3.194136,101.633640
3,Bandar Sri Permaisuri,0.00,2,3.100205,101.718107
4,Bandar Tasik Selatan,0.01,0,3.076097,101.711447


In [128]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(65, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
32,Lembah Pantai,0.02,0,3.106370,101.669919
28,"Kampung Baru, Kuala Lumpur",0.01,0,3.164949,101.706859
29,Kampung Datuk Keramat,0.02,0,3.168953,101.725970
63,Titiwangsa,0.01,0,3.173201,101.695923
33,Maluri,0.02,0,3.123294,101.726938
34,Medan Tuanku,0.02,0,3.159258,101.698911
35,Miharja,0.02,0,3.120931,101.717861
41,Segambut,0.02,0,3.187148,101.664489
42,Semarak,0.01,0,3.189566,101.722474
43,Sentul Raya,0.01,0,3.176862,101.690442


In [129]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 8. Examine Clusters

In [0]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [131]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
32,Lembah Pantai,0.02,0,3.106370,101.669919
28,"Kampung Baru, Kuala Lumpur",0.01,0,3.164949,101.706859
29,Kampung Datuk Keramat,0.02,0,3.168953,101.725970
63,Titiwangsa,0.01,0,3.173201,101.695923
33,Maluri,0.02,0,3.123294,101.726938
34,Medan Tuanku,0.02,0,3.159258,101.698911
35,Miharja,0.02,0,3.120931,101.717861
41,Segambut,0.02,0,3.187148,101.664489
42,Semarak,0.01,0,3.189566,101.722474
43,Sentul Raya,0.01,0,3.176862,101.690442


In [132]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
13,Bukit Kiara,0.0,2,3.158462,101.636003
62,Taman Wahyu,0.0,2,3.214504,101.672211
22,Desa Petaling,0.0,2,3.084185,101.703552
3,Bandar Sri Permaisuri,0.0,2,3.100205,101.718107
58,Taman P. Ramlee,0.0,2,3.194460,101.709069
57,Taman OUG,0.0,2,3.075488,101.670810
56,Taman Midah,0.0,2,3.100050,101.731972
55,Taman Melati,0.0,2,3.219527,101.721867
54,Taman Len Seng,0.0,2,3.072849,101.744999
53,Taman Ibukota,0.0,2,3.211147,101.715124


### Observations:

Most of the shopping malls are concentrated in the central area of Kuala Lumpur city, with the highest number in cluster 2 and moderate number in cluster 0. On    the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to      open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense          competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.
